In [ ]:
import torch

# 模拟数据：3 个样本 × 5 个目标
all_preds = torch.tensor([[0.1396, 0.0394, 0.2231, 0.1790, 0.4021],
                          [0.1501, -0.0556, 0.3123, 0.0945, 0.4067],
                          [0.1046, -0.0863, 0.2008, 0.0183, 0.2191]], device='cuda:0', dtype=torch.float16)

all_targets = torch.tensor([[16.2751, 0.0000, 31.9984, 16.2750, 48.2735],
                            [24.2376, 0.0000, 30.9703, 24.2376, 55.2079],
                            [32.1910, 23.0755, 2.6135, 55.2665, 57.8800]], device='cuda:0', dtype=torch.float32)

# 打印查看这两个张量
print("all_preds:", all_preds)
print("all_targets:", all_targets)


In [ ]:
import torch
import torch.nn as nn

def compute_loss(preds, targets):
    """
    与 Kaggle 官方 Global Weighted R² 对齐的损失：
    loss = ss_res / ss_tot
    （越小越好，与官方评分方向一致）
    """
    # 确保 preds 和 targets 是二维张量（[B, 5]），如果不是就 reshape
    preds = preds.view(-1, 5)  # [3, 5]
    targets = targets.view(-1, 5)  # [3, 5]

    # 权重定义（与 Kaggle 官方相同）
    weights = torch.tensor([0.1, 0.1, 0.1, 0.2, 0.5], device=preds.device)

    # 拼接所有目标
    y_true_flat = targets.view(-1)  # 展平真实值
    y_pred_flat = preds.view(-1)  # 展平预测值
    
    # 生成与目标一样大小的权重张量，并重复权重，确保每个目标都有对应的权重
    w_flat = torch.cat([torch.full_like(targets[:, i], weights[i], device=preds.device) for i in range(5)])

    # 全局加权均值
    y_mean = torch.sum(w_flat * y_true_flat) / torch.sum(w_flat)

    # 打印一些调试信息
    print(f"y_true_flat shape: {y_true_flat.shape}")
    print(f"y_pred_flat shape: {y_pred_flat.shape}")
    print(f"w_flat shape: {w_flat.shape}")

    # 加权残差平方和 与 总平方和
    ss_res = torch.sum(w_flat * (y_true_flat - y_pred_flat) ** 2)
    ss_tot = torch.sum(w_flat * (y_true_flat - y_mean) ** 2)

    print(f"ss_res: {ss_res}")
    print(f"ss_tot: {ss_tot}")

    # 损失定义：直接取残差比
    loss = ss_res / ss_tot

    print(f"Loss: {loss}")
    return loss

# 计算并打印损失
loss = compute_loss(all_preds, all_targets)
print(f"Computed loss: {loss.item()}")
